<a href="https://colab.research.google.com/github/NiyaziOnurYantira/FineTuning/blob/main/Inference_tr_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [3]:
MODEL_NAME = "OnurYantira/llama3-8b-turkish-english-medical-merged"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [5]:
system_prompt = "You are a helpful medical assistant. Provide concise, accurate, and professional answers to the user's health-related questions."
user_message = "A patient is experiencing fatigue, brittle nails, and pale skin. What could be the possible cause and what tests would you recommend?"

In [6]:
prompt = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    f"{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    f"{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
)

In [7]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

In [12]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = generated_text.split("<|start_header_id|>assistant<|end_header_id|>\n")[-1].strip()

In [13]:
print("🔎 Model Yanıtı:")
print(response)

🔎 Model Yanıtı:
system
You are a helpful medical assistant. Provide concise, accurate, and professional answers to the user's health-related questions.user
A patient is experiencing fatigue, brittle nails, and pale skin. What could be the possible cause and what tests would you recommend?assistant
Based on the symptoms of fatigue, brittle nails, and pale skin, anemia (a condition where there isn't enough healthy red blood cells in your body) might be a possible underlying cause. Here are some suggested laboratory tests: 

1. Complete Blood Count (CBC): This test measures different parts of your blood and its ability to carry oxygen throughout your body.
2. Iron Test: A low iron level may indicate iron deficiency anemia.
3. Ferritin Test: This test measures the amount of stored iron in your body.
4. Folate Test: Deficiency in folate can lead to megaloblastic anemia.
5. Vitamin B12 Test: Low levels of vitamin B12 may also contribute to anemia.

These tests will help diagnose if anemia is

In [14]:
system_prompt = "Sen Türkçe konuşan yardımcı bir medikal asistansın. Kullanıcının sağlıkla ilgili sorularına açık, doğru ve profesyonel yanıtlar ver."
user_message = "Bir hastada halsizlik, solgunluk ve tırnaklarda kırılma şikayetleri var. Bu durum neye işaret edebilir ve hangi testler yapılmalıdır?"

# 🔧 LLaMA 3.1 chat formatına uygun prompt
prompt = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    f"{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    f"{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
)

# 🔄 Tokenize ve modele gönder
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
model.to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

# ✂️ Yanıtı ayıkla ve göster
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = generated_text.split("<|start_header_id|>assistant<|end_header_id|>\n")[-1].strip()

print("🔍 Model Yanıtı (Türkçe):")
print(response)

🔍 Model Yanıtı (Türkçe):
system
Sen Türkçe konuşan yardımcı bir medikal asistansın. Kullanıcının sağlıkla ilgili sorularına açık, doğru ve profesyonel yanıtlar ver.user
Bir hastada halsizlik, solgunluk ve tırnaklarda kırılma şikayetleri var. Bu durum neye işaret edebilir ve hangi testler yapılmalıdır?assistant
Bu belirtiler demir eksikliği anemisi, vitamin B12 eksikliği veya folat eksikliği gibi beslenme bozukluğu kaynaklı olabilir.
